In [5]:
import numpy as np
from helper import Particle, Cell, build_tree

In [6]:
n = 100
weight = 1 / n
particles = [Particle(w = weight) for i in range(n)]
capacity = 10 # max number of particles in a single cell

In [7]:
root = Cell(capacity)
root.x, root.y, root.z = 0.5, 0.5, 0.5
root.r = 0.5

In [8]:
cells = build_tree(particles, root, capacity)

particle 0 is stored in cell 0
particle 1 is stored in cell 0
particle 2 is stored in cell 0
particle 3 is stored in cell 0
particle 4 is stored in cell 0
particle 5 is stored in cell 0
particle 6 is stored in cell 0
particle 7 is stored in cell 0
particle 8 is stored in cell 0
particle 9 is stored in cell 0
======start the split of cell 0======
+++cell 1 is created as a child of cell 0
>>>particle 0 is reallocated in cell 1
+++cell 2 is created as a child of cell 0
>>>particle 1 is reallocated in cell 2
>>>particle 2 is reallocated in cell 1
+++cell 3 is created as a child of cell 0
>>>particle 3 is reallocated in cell 3
+++cell 4 is created as a child of cell 0
>>>particle 4 is reallocated in cell 4
>>>particle 5 is reallocated in cell 1
>>>particle 6 is reallocated in cell 3
>>>particle 7 is reallocated in cell 3
+++cell 5 is created as a child of cell 0
>>>particle 8 is reallocated in cell 5
>>>particle 9 is reallocated in cell 1
======end split cell 0======
+++cell 6 is created as

In [9]:
def get_leaf_cell_multipole(particles, p, cells, capacity):
    # if the current cell p is not a leaf cell, then recursively traverse down
    if cells[p].nleaf >= capacity:
        for c in range(8):
            if cells[p].clist & (1 << c): # if the child exists
                get_leaf_cell_multipole(particles, cells[p].child[c], cells, capacity)
    # otherwise cell p is a leaf cell
    else:
        # loop in leaf particles
        for i in range(cells[p].nleaf):
            l = cells[p].leaf[i]
            dx, dy, dz = cells[p].x-particles[l].x, cells[p].y-particles[l].y, cells[p].z-particles[l].z
            cells[p].multipole += particles[l].w * np.array((1, dx, dy, dz,\
                                               dx**2/2, dy**2/2, dz**2/2,\
                                               dx*dy/2, dy*dz/2, dz*dx/2)) 

In [10]:
root = 0
get_leaf_cell_multipole(particles, root, cells, capacity)

In [11]:
# calculate parent's multilope based on its children's multipole
def get_parent_cell_multipole(p, c, cells):
    dx, dy, dz = cells[p].x-cells[c].x, cells[p].y-cells[c].y, cells[p].z-cells[c].z
    
    Dxyz =  np.array((dx, dy, dz))
    Dyzx = np.roll(Dxyz,-1) #It permutes the array (dx,dy,dz) to (dy,dz,dx) 
    
    cells[p].multipole += cells[c].multipole
    
    # dipole
    cells[p].multipole[1:4] += cells[c].multipole[0] * Dxyz
    
    # quadrop
    cells[p].multipole[4:7] += cells[c].multipole[1:4] * Dxyz\
                             + 0.5*cells[c].multipole[0] *  Dxyz**2
    
    cells[p].multipole[7:] += 0.5*np.roll(cells[c].multipole[1:4], -1) * Dxyz \
                            + 0.5*cells[c].multipole[1:4] * Dxyz \
                            + 0.5*cells[c].multipole[0] * Dxyz * Dyzx   

In [ ]:
def upward_sweep(cells):
    for c in range(len(cells)-1, 0, -1):
        p = cells[c].parent
        get_parent_cell_multipole(p, c, cells)